In [66]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize,word_tokenize
import re
from nltk.corpus import stopwords
import nltk
import numpy as np

In [67]:
# Loading clik_list 
url_data=pd.read_excel(r"C:\Mba notes\novels\Intern Dataset\Text Analysis\cik_list.xlsx")

#loading master data
master_data=pd.read_csv(r"C:\Mba notes\novels\Intern Dataset\Text Analysis\LoughranMcDonald_MasterDictionary_2018.csv")

#loading uncertainity and data 
uncertainity=pd.read_excel(r"C:\Mba notes\novels\Intern Dataset\Text Analysis\uncertainty_dictionary.xlsx")
constraining_dictionary=pd.read_excel(r"C:\Mba notes\novels\Intern Dataset\Text Analysis\constraining_dictionary.xlsx")

# making list of all url
url=[]
for i in url_data["SECFNAME"]:
    url.append("https://www.sec.gov/Archives/"+i)
    
# getting words from dictionary 
Dict_words=[i for i in master_data["Word"]]
Dict_words=str(Dict_words)
Dict_words=Dict_words.lower()

# getting positive words
df1=master_data[["Word","Positive","Negative"]]
positive_words=[]
for i,j in zip(df1["Word"],df1["Positive"]):
    if j!=0:
        positive_words.append(i)
positive_words=str(positive_words)
positive_words=positive_words.lower()

# getting negative words
negative_words=[]
for i,j in zip(df1["Word"],df1["Negative"]):
    if j!=0:
        negative_words.append(i)
negative_words=str(negative_words)
negative_words=negative_words.lower()

# getting text from url
def scrap(url):
    u=url
    webpage=requests.get(u)
    soup=BeautifulSoup(webpage.text,"html.parser")
    return soup
txt1=scrap(url[0])
txt1=str(txt1)
txt2=txt1.lower()



In [ ]:
positive_score1=[]
negative_score1=[]
polarity_score1=[]
average_sentence_length1=[]
percentage_of_complex_words1=[]
fog_index1=[]
complex_word_count1=[]
word_count1=[]
uncertainty_score1=[]
constraining_score1=[]
positive_word_proportion1=[]
negative_word_proportion1=[]
uncertainty_word_proportion1=[]
constraining_word_proportion1=[]

In [ ]:
for i in range(0-20):

        txt1=scrap(url[i])
        txt1=str(txt1)
        txt2=txt1.lower()
        #cleaning the text
        c1=re.sub("\d","",txt2)
        c1=re.sub("\W"," ",c1)
        c1=res = re.sub(' +', ' ', c1)
        res=re.sub(r"[^\w\s]","",c1)
        res=re.sub("www","",res)
        res=re.sub("______","",res)

        # tokenization
        tokens=word_tokenize(res)
        w=[]

        #Personal Pronouns
        P=['I', 'we',' my',' ours','us']
        Personal_Pronouns=[]
        for i in P:
            if i in txt1:
                Personal_Pronouns.append(1)

        #removing stopwords
        for word in tokens:
                if not word in stopwords.words("english"):
                    w.append(word)

        # removing words greater than length 15 and less than 2
        for i in w:
            if len(i)>15 or len(i)<2:
                w.remove(i)

        # taking only those words which are in dictionary and our cleaned word
        word=[]
        for i in w:
            if i in Dict_words:
                word.append(i)


        # lemmatizing based on pos_tag
        pos_tag=nltk.pos_tag(word)
        lemma=WordNetLemmatizer()
        for i in range(len(pos_tag)):
            if pos_tag[i][1]=='RB':
                    lemma.lemmatize(word[i],pos="a")
            elif pos_tag[i][1]=='VBG':
                lemma.lemmatize(word[i],pos="v")
            elif pos_tag[i][1]=="NN":
                lemma.lemmatize(word[i],pos="n")
            else:          
                lemma.lemmatize(word[i])


        # calculating Positive score
        positive_score=[]
        for i in word:
            if i in positive_words:
                positive_score.append(1)
        positive_score=sum(positive_score)

        # Calculating Negative score
        negative_score=[]
        for i in word:
            if i in negative_words:
                negative_score.append(1)
        negative_score=sum(negative_score)

        #Calculating Polarity score + Word count
        Polarity_Score = (positive_score-negative_score)/ ((positive_score + negative_score) + 0.000001)
        Polarity_Score 
        Word_Count=len(word)

        # Syllable count per word + complex word count
        vowels=["a","e","i","o","u"]
        no=["es","ed"]
        Complex_word_count=[]
        for i in word:
            Syllable_Count_Per_Word=[]
            for j in i:
                if j in vowels and j not in no:
                    Syllable_Count_Per_Word.append(1)
            if sum(Syllable_Count_Per_Word)>2:
                Complex_word_count.append(1)
        Complex_word_count=sum(Complex_word_count)


        # Calculationg Average Sentence length
        sent=sent_tokenize(txt2)
        avg_sentence_length=len(word)/len(sent)

        #Percentage_of Complex words
        Percentage_of_Complex_words = Complex_word_count/len(word) 

        #Fog_index
        Fog_Index= 0.4*(avg_sentence_length + Percentage_of_Complex_words)

        #9Average Word Length
        Average_Word_Length=round(sum([len(i) for i in tokens])/len(tokens),2)
        Average_Word_Length

        # Calculating uncertaininty score 
        uncertainity_score=[]
        for i in word:
            if i in str(uncertainity["Word"]).lower():
                uncertainity_score.append(1)
        uncertainity_score=sum(uncertainity_score)

        # Calculating constraining score
        constraining_score=[]
        for i in word:
            if i in str(constraining_dictionary["Word"]).lower():
                constraining_score.append(1)
        constraining_score=sum(constraining_score)

        #positive word proportion
        positive_word_prportion=positive_score/len(word)

        #negative word proportion
        negative_word_proportion=negative_score/len(word)

        #constraining word proportion
        constraining_word_prportion=constraining_score/len(word)

        #uncertainity word proportion
        uncertainity_word_prportion=uncertainity_score/len(word)


        #apending all the values inside the variables
        positive_score1.append(positive_score), negative_score1.append(negative_score), polarity_score1.append(Polarity_Score),
        average_sentence_length1.append(avg_sentence_length), percentage_of_complex_words1.append(Percentage_of_Complex_words),
        fog_index1.append(Fog_Index), complex_word_count1.append(Complex_word_count), word_count1.append(Word_Count),
        uncertainty_score1.append(uncertainity_score), constraining_score1.append(constraining_score),
        positive_word_proportion1.append(positive_word_prportion), negative_word_proportion1.append(negative_word_proportion),
        uncertainty_word_proportion1.append(uncertainity_word_prportion), constraining_word_proportion1.append(uncertainity_word_prportion)


In [48]:
variables=[positive_score1,
    negative_score1,
    polarity_score1,
    average_sentence_length1,
    percentage_of_complex_words1,
    fog_index1,
    complex_word_count1,
    word_count1,
    uncertainty_score1,
    constraining_score1,
    positive_word_proportion1,
    negative_word_proportion1,
    uncertainty_word_proportion1,
    constraining_word_proportion1]
output=pd.DataFrame(variables)
output=output.T
c=output
output.to_csv(r"C:\Mba notes\novels\Intern Dataset\Text Analysis\output_files\output(0-20).csv")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,12504.0,44303.0,-0.559773,326.216092,0.090709,130.522720,12872.0,141904.0,57.0,1798.0,0.088116,0.312204,0.000402,0.000402
1,40.0,137.0,-0.548023,29.666667,0.366292,12.013184,163.0,445.0,16.0,25.0,0.089888,0.307865,0.035955,0.035955
2,49039.0,164412.0,-0.540513,328.364300,0.089943,131.381697,44508.0,494845.0,275.0,7583.0,0.099100,0.332249,0.000556,0.000556
3,8005.0,28566.0,-0.562221,335.941176,0.086795,134.411189,7931.0,91376.0,53.0,1210.0,0.087605,0.312620,0.000580,0.000580
4,68018.0,153829.0,-0.386803,65.591516,0.076134,26.267060,19071.0,250494.0,82626.0,50541.0,0.271535,0.614103,0.329852,0.329852
5,33023.0,83658.0,-0.433961,126.529464,0.110914,50.656151,15718.0,141713.0,32543.0,23329.0,0.233027,0.590334,0.229640,0.229640
6,174.0,580.0,-0.538462,2.453771,0.524046,1.191127,1057.0,2017.0,39.0,46.0,0.086267,0.287556,0.019336,0.019336
7,172.0,513.0,-0.497810,2.419589,0.507246,1.170734,1015.0,2001.0,49.0,59.0,0.085957,0.256372,0.024488,0.024488
8,998.0,4339.0,-0.626007,22.169211,0.552999,9.088884,9636.0,17425.0,170.0,298.0,0.057274,0.249010,0.009756,0.009756
9,250.0,1388.0,-0.694750,21.735043,0.571766,8.922723,2908.0,5086.0,48.0,71.0,0.049155,0.272906,0.009438,0.009438


In [97]:
import os
path="C:/Mba notes/novels/Intern Dataset/Text Analysis/output_files"
join=os.listdir(path)
final_path=[]
for i in join:
    fp=os.path.join(path,i)
    final_path.append(fp)
dfs=[]
for i in final_path:
    f=pd.read_csv(i)
    dfs.append(f)
data=pd.concat(dfs,ignore_index=True)
output=pd.read_csv(r"C:\Mba notes\novels\Intern Dataset\Text Analysis\output.csv")
output_data_stucture=pd.read_excel(r"C:\Mba notes\novels\Intern Dataset\Text Analysis\Structure.xlsx")
output_data_stucture=output_data_stucture.drop(["CIK","CONAME","FYRMO","FDATE","FORM","SECFNAME","constraining_words_whole_report"],1)
output.columns=output_data_stucture.columns
final_output=pd.concat([url_data,output],1)
final_output["constraining_words_whole_report"]=final_output["word_count"]/final_output["constraining_score"]
final_output.to_csv(r"C:\Mba notes\novels\Intern Dataset\Text Analysis\final_output_data.csv")